#### Before run the .ipynb file, please follow the link and download AVA image database first.
#### Link to download AVA: http://academictorrents.com/details/71631f83b11d3d79d8f84efe0a7e12f0ac001460
#### Note: Need a Bittorrent client to download all the files
#### After finishing downloading, we have an folder called "AVA_dataset" which contains all the 7z files of images
#### Unzip all the files to the folder "AVA_dataset"
#### Move the AVA_dataset which contains all AVA images to the folder "database" contained in our project

## Program Start ...

In [69]:
import os
import tarfile
import shutil
import glob
import requests
import zipfile
import io
import urllib
import cv2
import pandas as pd
import sys
from sys import stdout

In [70]:
if sys.version_info[0] < 3:
    raise Exception("Must be using Python 3")

In [71]:
from urllib.request import urlretrieve

### Download VOC ...

In [87]:
url_voc = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
voc, headers = urllib.request.urlretrieve(url_voc, 'voc.tar')

In [91]:
tar = tarfile.open('voc.tar')
tar.extractall('database/voc_emotic_ava')

In [92]:
files = "database/voc_emotic_ava/VOCdevkit/VOC2012/JPEGImages/*"
for path in glob.glob(files):
    name = path.split("/")[-1]
    newp1 = path.replace(name,"")+"JPEGImages__"+name
    newp2 = path.replace(name, "")+"VOC"+name
    os.rename(path, newp1)
    shutil.copy(newp1, 'database/voc_emotic_ava/')
    os.rename(newp1, newp2)
    shutil.copy(newp2, 'database/voc_emotic_ava/')
    os.remove(newp2)
shutil.rmtree('database/voc_emotic_ava/VOCdevkit')

### Download Emotic...

In [93]:
dest = os.path.dirname("database/voc_emotic_ava")
session = requests.Session()
response = session.get('https://docs.google.com/uc?export=download',params={'id': '0B7sjGeF4f3FYQUVlZ3ZOai1ieEU'}, stream=True)
token = None
for key, value in response.cookies.items():
    if key.startswith('download_warning'):
        token = value
if token:
    params = {'id': '0B7sjGeF4f3FYQUVlZ3ZOai1ieEU', 'confirm': token}
    response = session.get('https://docs.google.com/uc?export=download', params=params, stream=True)
current_download_size = [0]
with open('emotic.zip', 'wb') as f:
    for chunk in response.iter_content(32768):
        f.write(chunk)

In [94]:
zipfile.ZipFile('emotic.zip', 'r').extractall('database/voc_emotic_ava')
folders = os.listdir("database/voc_emotic_ava/emotic/")
for folder in folders: 
    files = "database/voc_emotic_ava/emotic/"+folder + "/images/*"
    for path in glob.glob(files):
        name = path.split("/")[-1]
        newp = path.replace(name,"")+"EMOTIC__"+name
        os.rename(path, newp)
        shutil.move(newp, 'database/voc_emotic_ava/')
shutil.rmtree('database/voc_emotic_ava/emotic')
shutil.rmtree('database/voc_emotic_ava/__MACOSX')

### Download Blur Detection Dataset ...

In [95]:
url_blur = "http://www.cse.cuhk.edu.hk/~leojia/projects/dblurdetect/data/BlurDatasetImage.zip"
blur, headers = urllib.request.urlretrieve(url_blur, 'blur.zip')
zipfile.ZipFile('blur.zip','r').extractall('database/blur_dataset')
for path in glob.glob('database/blur_dataset/image/*'):
    shutil.move(path, 'database/blur_dataset/')
shutil.rmtree('database/blur_dataset/image')

### Rename and Move AVA Images

In [96]:
if os.path.isdir('database/AVA_dataset') == True:
    files = 'database/AVA_dataset/images/*'
    for path in glob.glob(files):
        name = path.split("/")[-1]
        newp = path.replace(name, "")+"AVA__"+name
        os.rename(path, newp)
        shutil.move(newp, 'database/voc_emotic_ava/')
    shutil.rmtree('database/voc_emotic_ava/AVA_dataset')

### Clean Image Not Been Used ...

In [97]:
df_ims = pd.read_csv('image_attri.csv')
ims = df_ims['name']
ims = ims.values.tolist()

In [98]:
folders = os.listdir('database')

In [99]:
for folder in folders:
    files = 'database/'+folder+"/*"
    for path in glob.glob(files):
        name = path.replace("database/","")
        if name not in ims:
            os.remove(path)

### Sampling Patches from Images

In [101]:
try:
    if os.path.isdir('database/patches') == False:
        os.mkdir('database/patches')
except OSError:
    print ("Creation of the directory %s failed" % 'patch')

In [102]:
def patch_sampling(im_path, patch_index, x, y, width, height):
    img=cv2.imread(im_path)
    patch = img[x:x+width, y:y+height]
    name = im_path.split('/')[-1]
    p_path = 'database/patches/'+name+"_"+"patch_"+str(patch_index)+".jpg"
    cv2.imwrite(p_path,patch)

In [103]:
df_coor = pd.read_csv('all_patches.csv')

In [104]:
for folder in folders:
    if folder != 'patches':
        im_paths = 'database/'+folder+"/*"
        for im_path in glob.glob(im_paths):
            im_name = im_path.split('/')[-1]
            patch_name = "patches/"+im_name
            row = df_coor.loc[df_coor['name_patch']==patch_name]
            if len(row) == 1:
                x1 = int(row['top_patch_1'].values.tolist()[0])
                y1 = int(row['left_patch_1'].values.tolist()[0])
                x2 = int(row['top_patch_2'].values.tolist()[0])
                y2 = int(row['left_patch_2'].values.tolist()[0])
                x3 = int(row['top_patch_3'].values.tolist()[0])
                y3 = int(row['left_patch_3'].values.tolist()[0])
                h1 = int(row['height_patch_1'].values.tolist()[0])
                w1 = int(row['width_patch_1'].values.tolist()[0])
                h2 = int(row['height_patch_2'].values.tolist()[0])
                w2 = int(row['width_patch_2'].values.tolist()[0])
                h3 = int(row['height_patch_3'].values.tolist()[0])
                w3 = int(row['width_patch_3'].values.tolist()[0])
                patch_sampling(im_path, 1, x1, y1, w1, h1)
                patch_sampling(im_path, 2, x2, y2, w2, h2)
                patch_sampling(im_path, 3, x3, y3, w3, h3)